# Marketing campaigns Subscription Predictor
Shiying Wang,  Karlos Muradyan, Gaurav Sinha

# Introduction

Businesses provide numerous services to their clients. It becomes important for them to know whether their clients need those services. Banking is one sector which provides numerous services to their clients. If a client subscribes to their service, it increases revenue for the bank. One such service offered by banks is *Term Deposit*. A term deposit is a form of deposit in which money is held for a fixed duration of time with the financial institution. A client will subscribe to a term deposit or not is dependent on a large number of features of a client. Banks generally have this information of clients which can help to predict whether a client will subscribe to a term deposit or not. This is an interesting problem which can be solved by analysing the data and building model to predict such clients behaviour. 

In this project, we will analyze a *Bank Marketing* data of a Portuguese banking institution, and predict whether the a client will subscribe for a term deposit from the marketing campaign. 


# Methods

## Data

The dataset we chose is Bank Marketing data of a Portugese banking instituion collected from [UCI Machine learning Repository](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing). The dataset is publicly available for research, the details of which are described in [Moro et al., 2014](#Reference). Detail of the source is provided in the reference session.

The dataset has 4521 observations, 16 features and a response variable `y` which indicates if the client subscribed a term deposit or not (binary: 'yes','no'). The classes are imbalance: there are 4000 no and 521 yes. There are 0 observations with missing values in the data set. For feature description, please refer to the [Attribute Information](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#).

## Exploratory Data Analysis

We split the data as 80% training dataset and 20% test data. We used the training dataset to perform the analysis. Figure 1 is the proportion of the response variable y. 

<img src="../reports/proportion_of_class.png">

Figure 1. Proportion of response variable: whether the client subscribed.

From the plot, we can see that the classes are highly imbalance: around 90% of people did not subscribe after the marketing campaign, and only around 10% subscribed after the marketing campaign. The highly imbalance dataset would cause the problem that the model only predicted the majority classes. We will discuss how to solve it in the [Model](#Model) section.

This dataset has combination of numerical and categorical features. We explored them respectively. Figure 2 Figure 3 shows the relationship between the numerical features. 

<img src="../reports/kendall_corr_matrix.png" width="400" height="400"> <img src="../reports/pearson_corr_matrix.png" width="400" height="400">

Figure 2: kendall and pearson correlation matrix for numerical variables

From the kendall and pearson correlation matrix, most features does not have correlation with others. The only big correlation between numerical features is between pdays and previous. Kendall's correlation coefficient gives value greater than 0.9

<img src="../reports/density_plot.png">
Figure 3: density plots of numerical features: pdays and duration

From Figure 3, we found some interested patterns: for people who did not subscribed (blue), the density plot for `duration` is more concentrated on the left. This indicates that this class tend to has lower duration of last contact. Most of the people who subscribed after the campaign (red) haven't had previous contacts before. Those indicates that `pdays` and `duration` might be important features.

## Model

In data preprocessing, we standardize the numerical features and use one-hot-encoding to convert categorical features into dummy variables. We used some traditional classification models for dealing with this classification problem such as `logistic regression`, `support vector machine` and `random forest` from `sklearn` package. To deal with the data imbalance, we tuned the hyper-parameter `class_weight` in built-in sklearn model parameter. Apart from that, we used one of the variations of Gradient Boosting Trees: LightGBM, as more complex solution to the problem.

We used F1 score as our main evaluation metric. Given the formula $F1_{score} = 2 * \frac{(precision * recall)}{(precision + recall)}$, it will try to find a balance between the precision and recall, which is very useful in our imbalance case.

To see the list of parameters used for each model in `GridSearchCV`, please see Appendix.

# Results & Discussion

## Results

We trained different variations of the above mentioned `sklearn` models. The performance of all models tested with `GridSearchCV` can be seen in the below chart:

<img src="../reports/training_report.png" width="600" height="600">
Figure 4: Training report

**Please note**, that the results shown are from `GridSearchCV` output itself, so `test_scores` is actually lower than what is displayed. `GridSearchCV` approximates test error using cross-validation technique and in practice overestimates it (gives higher score than actually).

Apart from that, we also show results in `csv` file. Here, all the results are from true `train` and `test` datasets and can be compared fairly. Here, we also show results of `LightGBM` model.

|   | Model name             | Best parameters                                                                                                                                                       | Train F1            | Test F1                     | Test accuracies            |
|---|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------|-----------------------------|----------------------------|
| 0 | LogisticRegression     | 'C': 0\.1, <br> 'penalty': 'l1',<br> 'solver': 'liblinear'                                                                                                             | 0\.521 | 0.493 | 0.832 |
| 1 | SVC                    | 'C': 0\.1,<br> 'degree': 3,<br> 'kernel': 'poly'"                                                                                                                        | 0\.5626 | 0\.533  | 0\.86 |
| 2 | RandomForestClassifier | 'criterion': 'entropy',<br> 'max\_depth': 16, <br>'n\_estimators': 25                                                                                                   | 0\.972 | 0\.376  | 0\.886 |
| 3 | Booster                | 'learning\_rate': 0\.1, <br>'lambda\_l1': 0\.5, <br>'max\_depth': 64,<br> 'num\_leaves': 32,<br> 'bagging\_fraction': 0\.9, <br>'bagging\_freq': 3,<br> 'bagging\_seed': 42,<br> 'seed': 42 | 0\.622 | 0\.413        | 0\.899        |

Table 1: Best results of all models


As we can see, the best Test `F1` score gave SVM, even though the best test accuracy has `LightGBM`. This is the result of class imbalance problem. As `F1` metric handles the best that problem, we chose `SVM` as our best model. For more exploration of the results, below we also show the confusion matrices of those 4 models:

<img src='../reports/confusion_matrix.png'>

## Limitation

In this analysis the best test `F1` score achieved is 0.533 with `SVM` which could be further improved by overcoming some of the limitations considered during analysis.`Booster` works well with imbalanced data sets. It has many hyperparameters which can be tuned to accomplish better result. Due to lack of computational resources and time only a small subset of hyperparameters are tuned. This opens up opportunity to improve the model further in future.The data set used in the analysis is imbalanced. Out of many ways to deal with this issue only two methods, choosing the right test statistic(F1-score) and giving higher weights to the less abundant data has been tried. Other methods or their combination can also be tried given more time. 

## Further directions

For future implementation, we can do further feature selection and feature engineering for better model performance. Also, we can implement some other ways to deal with the imbalance dataset, such as oversampling (SMOTE) and down-sampling methods.One interesting approach to deal with this is clustering the abundant class and only taking the median value for that class.This can remove correlated abundant example and reduce imbalance. We are keen on using these different approaches or trying combination of these to further improve the performance. `Booster` is known to work well on imbalanced data set, so we can explore other hyperparameters and greater range of values of these hyperparameters for improving the results.

## Appendix

### GridSearchCV search:

As discussed in earlier sections, we used `GridSearchCV` implementation of `sklearn` package to find the best hyperparameters for our models. We tried several values of hyperparameters for `Logistic Regression`, `SVM` and `Random Forest`. The tested ranges of parameters can be found in the table below:

|        Model        | Tested Parameters |  Range |
|---------------------|:-----------------:|------:|
| Logistic Regression | solver            | 'saga', 'liblinear', 'lbfgs', 'newton-cg', 'sag' |
|                     |  penalty          | 'l1', 'l2' |
|                     |  C                | 0.01, 0.1, 1, 10 |
|         SVM         |  kernel           | 'rbf', 'poly' |
|                     |deg. for poly kern.| 2, 3, 4 |
|                     |  C                | 0.01, 0.1, 1, 10 |
|    Random Forest    |  n_estimators     |    25, 50, 75 |
|                     |  max_depth        | None, 16, 32 |
|                     |  criterion        | 'gini', 'entropy'|

Table 2. Hyperparameters of all models tested with `GridSearchCV`

# Reference


[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

UCI Machine Learning Repository. University of California, Irvine, School of Information; Computer Sciences.https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Raschka, S. and Mirjalili, V., 2019. Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow 2. Packt Publishing Ltd.

Bisong, Ekaba. 2019. “Matplotlib and Seaborn.” In Building Machine Learning and Deep Learning Models on Google Cloud Platform, 151–65. Springer.

Chandra, Rakesh Vidya, and Bala Subrahmanyam Varanasi. 2015. Python Requests Essentials. Packt Publishing Ltd.

Costa-Luis, Casper da. 2019. “Tqdm: A Fast, Extensible Progress Meter for Python and Cli.” Journal of Open Source Software 4 (37): 1277.

de Jonge, Edwin. 2018. Docopt: Command-Line Interface Specification Language. https://CRAN.R-project.org/package=docopt.

Ke, Guolin, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. “Lightgbm: A Highly Efficient Gradient Boosting Decision Tree.” In Advances in Neural Information Processing Systems, 3146–54.

Keleshev, Vladimir. 2014. Docopt: Command-Line Interface Description Language. https://github.com/docopt/docopt.

McKinney, Wes, and others. 2011. “Pandas: A Foundational Python Library for Data Analysis and Statistics.” Python for High Performance and Scientific Computing 14 (9).

Oliphant, Travis E. 2006. A Guide to Numpy. Vol. 1. Trelgol Publishing USA.

Pedregosa, Fabian, Gaël Varoquaux, Alexandre Gramfort, Vincent Michel, Bertrand Thirion, Olivier Grisel, Mathieu Blondel, et al. 2011. “Scikit-Learn: Machine Learning in Python.” Journal of Machine Learning Research 12 (Oct): 2825–30.

R Core Team. 2019. R: A Language and Environment for Statistical Computing. Vienna, Austria: R Foundation for Statistical Computing. https://www.R-project.org/.

Reitz,Kenneth.2019._Requests: HTTP for Humans_.https://pypi.org/project/requests/.

Salunke, Sagar Shivaji. 2014. Selenium Webdriver in Python: Learn with Examples. CreateSpace Independent Publishing Platform.

Van Rossum, Guido, and Fred L. Drake. 2009. Python 3 Reference Manual. Scotts Valley, CA: CreateSpace.

VanderPlas, Jacob, Brian Granger, Jeffrey Heer, Dominik Moritz, Kanit Wongsuphasawat, Arvind Satyanarayan, Eitan Lees, Ilia Timofeev, Ben Welsh, and Scott Sievert. 2018. “Altair: Interactive Statistical Visualizations for Python.” Journal of Open Source Software 3 (32): 1057.

Wickham, Hadley. 2011. “Testthat: Get Started with Testing.” The R Journal 3: 5–10. https://journal.r-project.org/archive/2011-1/RJournal_2011-1_Wickham.pdf.